The purpose of this notebook is to demonstrate on a randomly selected diffusion weighted dataset how we can obtain a DTI fit, save it to a file, and save an FA image from it. The accompanying script `05.1_dti_fit.py` actually does this stuff to the full dataset. This notebook is just a demo to play with.

In [1]:
import os
import glob
import random
import json
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import dipy.io.image
import dipy.io
import dipy.core.gradients
import dipy.reconst.dti
import dipy.segment.mask

In [2]:
data_dir = '/home/ebrahim/data/abcd/DMRI_extracted'
img_dirs = glob.glob(os.path.join(data_dir,'*ABCD-MPROC-DTI*/sub-*/ses-*/dwi/'))
output_dir = './dti_fit_images/'

In [3]:
sampled_fmriresults01_df = pd.read_csv('01.0_abcd_sample/sampled_fmriresults01.csv')
sampled_fmriresults01_df['dirname'] = sampled_fmriresults01_df.derived_files.apply(lambda x : x.split('/')[-1].strip('.tgz'))
dirname_to_full_path = {img_dir.split('/')[-5]:img_dir for img_dir in img_dirs}

In [4]:
data = []
for (subjectkey,interview_age),df in sampled_fmriresults01_df.groupby(['subjectkey', 'interview_age']):
    paths = []
    for _,row in df.iterrows():
        if row.dirname not in dirname_to_full_path.keys():
            raise FileNotFoundError(f"Could not find a directory for fmriresults01 id {row.fmriresults01_id}")
        img_dir = dirname_to_full_path[row.dirname]
        dwi_path = glob.glob(os.path.join(img_dir, '*.nii'))[0]
        bval_path = glob.glob(os.path.join(img_dir, '*.bval'))[0]
        bvec_path = glob.glob(os.path.join(img_dir, '*.bvec'))[0]
        paths.append({
            'img_dir' : img_dir,
            'dwi_path' : dwi_path,
            'bval_path' : bval_path,
            'bvec_path' : bvec_path,
        })
    data.append({
        'paths' : paths,

        'subjectkey' : row.subjectkey,
        'interview_age' : row.interview_age,
    })

In [5]:
# Run this to look at the number of bvals for each image, alongside subject id and interview age

for d in data:
    for i,p in enumerate(d['paths']):
        bvals, bvecs = dipy.io.read_bvals_bvecs(p['bval_path'], p['bvec_path'])
        print(f"{len(bvals)} \t {d['subjectkey']} \t {d['interview_age']}",
              f"(part {i+1} of {len(d['paths'])})" if len(d['paths'])>1 else "")

103 	 NDAR_INV1306ZGW1 	 115 
103 	 NDAR_INV36E4JVZ2 	 129 
103 	 NDAR_INV36E4JVZ2 	 153 
104 	 NDAR_INV3XNH8EYM 	 138 
103 	 NDAR_INV4C8YJ9BG 	 131 
103 	 NDAR_INV4C8YJ9BG 	 157 
103 	 NDAR_INV52XG9LJ3 	 114 
103 	 NDAR_INV52XG9LJ3 	 138 
103 	 NDAR_INV5K67LYGB 	 127 
103 	 NDAR_INV5K67LYGB 	 151 
103 	 NDAR_INV5U9JTZ3X 	 113 
51 	 NDAR_INV6L7B1GX3 	 133 (part 1 of 2)
51 	 NDAR_INV6L7B1GX3 	 133 (part 2 of 2)
104 	 NDAR_INV72Z70LY1 	 136 
103 	 NDAR_INV7H3FYUBB 	 121 
103 	 NDAR_INV7H3FYUBB 	 146 
103 	 NDAR_INV7L2M29CY 	 114 
103 	 NDAR_INV7NVJNHRE 	 112 
104 	 NDAR_INV7NVJNHRE 	 136 
104 	 NDAR_INV7RGX9G26 	 109 
103 	 NDAR_INV93DMBG2D 	 111 
103 	 NDAR_INV9D6W29Z7 	 123 
104 	 NDAR_INV9D6W29Z7 	 146 
103 	 NDAR_INV9KX83G7Z 	 111 
103 	 NDAR_INV9KX83G7Z 	 135 
103 	 NDAR_INVB2FVBDTD 	 120 
103 	 NDAR_INVB2FVBDTD 	 144 
103 	 NDAR_INVB349E4D3 	 128 
103 	 NDAR_INVB349E4D3 	 152 
103 	 NDAR_INVBB9X243N 	 117 
103 	 NDAR_INVC624EF1K 	 109 
103 	 NDAR_INVC624EF1K 	 131 
103 	 NDAR_INVCX

Notice a few things here:
- There are scan sessions that got split up over multiple files, ~~maybe because there was a break in the middle of scanning~~ Actually it's because the Philips scanners always produce two image files, this is pointed out in the ABCD release notes (look for "ABCD Imaging Instruments", "Expected File Sets"). For those there are 51+51 = 102 total b-values, with one b=3000 image being missing compared to what is described in the [scanning protocol](https://abcdstudy.org/images/Protocol_Imaging_Sequences.pdf).
- Even among the non-split files, the total number of b-values varies among 103,104. When there are 104 b-values it seems that there's an extra b=0 image.

In [16]:
# Function to load data from one of the dictionaries listed in the object "data" defined above
def load_data(d):
    img_data_list =[]
    bvals_list = []
    bvecs_list = []
    prev_affine_transform = None

    for p in d['paths']:
        img_data, affine = dipy.io.image.load_nifti(p['dwi_path'])
        assert((prev_affine_transform is None) or (affine==prev_affine_transform).all())
        prev_affine_transform = affine  
        bvals, bvecs = dipy.io.read_bvals_bvecs(p['bval_path'], p['bvec_path'])
        img_data_list.append(img_data)
        bvals_list.append(bvals)
        bvecs_list.append(bvecs)
        bvals = np.concatenate(bvals_list)
    img_data = np.concatenate(img_data_list, axis=-1)
    bvecs = np.concatenate(bvecs_list, axis=0)
    gtab = dipy.core.gradients.gradient_table(bvals, bvecs)
    return img_data, affine, gtab

In [18]:
d_break = random.choice([d for d in data if len(d['paths'])>1]) # Pick one from the Philips kids
d_cts = random.choice([d for d in data if len(d['paths'])==1]) # Pick one from the single-file kids

img_data, affine, gtab = load_data(d_break) # Load one to demonstrate how we process it below

In [ ]:
img_data_masked, mask = dipy.segment.mask.median_otsu(img_data, vol_idx = range(img_data.shape[-1]))

In [ ]:
plt.imshow(mask[:,:,90].T, origin='lower')
plt.show()

In [ ]:
tensor_model = dipy.reconst.dti.TensorModel(gtab)

In [ ]:
tensor_fit = tensor_model.fit(img_data_masked)

In [ ]:
# If we wanted to obtain the FA image from the tensor_fit object, here is how to do that
fa = dipy.reconst.dti.fractional_anisotropy(tensor_fit.evals)

In [ ]:
np.isnan(fa).sum() # Notice there are no nans

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(20,10))
axs[0].imshow(fa[62,:,:].T, origin='lower', cmap='gray')
axs[1].imshow(fa[:,:,80].T, origin='lower', cmap='gray')
plt.show()

In [ ]:
# Save the lower triangular part, 
# i.e. the unique elements of the diffusion tensor in the order Dxx, Dxy, Dyy, Dxz, Dyz, Dzz
lt = tensor_fit.lower_triangular()
dipy.io.image.save_nifti('test_lt.nii.gz', lt, affine)

If you wanted to construct the FA image from the lower triangular part of the diffusion tensor, here's how:

In [ ]:
eig = dipy.reconst.dti.eig_from_lo_tri(lt) # has eigenvals and eigenvecs
eigvals = eig[:,:,:,:3] # take only the eigenvals
fa = dipy.reconst.dti.fractional_anisotropy(eigvals)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(20,10))
axs[0].imshow(fa[62,:,:].T, origin='lower', cmap='gray')
axs[1].imshow(fa[:,:,80].T, origin='lower', cmap='gray')
plt.show()